# Preliminary Data Analysis

In [38]:
from pyspark import SparkContext
from pyspark.sql import SQLContext, SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import *

sc = SparkContext('local')
spark = SparkSession(sc)

## Meteorological Data for Bennett's Point, Ace Basin 2004

In [18]:
data_sample = "./input/raw/ACE Basin/meteorological/data/Bennett's Point/"

aceb04_data = spark.read.csv(data_sample + "acebpmet2004.txt",header=True)
dropped_cols = [f for f in aceb04_data.columns if f[0] == 'F']
aceb04_data = aceb04_data.drop(*dropped_cols)

split_col = F.split(aceb04_data['DateTimeStamp'], '\ ')
aceb04_data = aceb04_data.withColumn('SMPLDATE', split_col.getItem(0)).withColumn('SMPLTIME', split_col.getItem(1))
aceb04_data = aceb04_data.drop('DateTimeStamp')

### Dimensions of 2004 data

In [19]:
print((aceb04_data.count(), len(aceb04_data.columns)))

(43924, 28)


## Meteorological Data for Bennett's Point, Ace Basin 2005

In [20]:
aceb05_data = spark.read.option("header", "true") \
    .option("delimiter", "\t") \
    .option("inferSchema", "true") \
    .csv(data_sample + "acebpmet2005.txt")
aceb05_data = aceb05_data.drop(*['STNCODE', 'USRCODES'])

### Dimensions of 2005 data

In [21]:
print((aceb05_data.count(), len(aceb05_data.columns)))

(44034, 28)


## Meteorological Data for Bennett's Point, Ace Basin 2006

In [22]:
aceb06_data = spark.read.option("header", "true") \
    .option("delimiter", "\t") \
    .option("inferSchema", "true") \
    .csv(data_sample + "acebpmet2006.txt")
aceb06_data = aceb06_data.drop(*['STNCODE', 'USRCODES'])

### Dimensions of 2006 data

In [23]:
print((aceb06_data.count(), len(aceb06_data.columns)))

(39043, 28)


## Compilation of ALL Meteorological Data for Bennett's Point, Ace Basin from 2004-2006

In [24]:
acebmet_data = aceb06_data.union(aceb04_data).union(aceb05_data)
acebmet_data.toPandas().to_csv("input/clean/agg/AceBasinMeteor_2004-2006.csv")

### Dimensions of All data

In [25]:
print((acebmet_data.count(), len(acebmet_data.columns)))

(127001, 28)


### Checking for Missing Data

In [52]:
from pyspark.sql.functions import isnan, when, count, col
acebmet_data.select([count(when(isnan(c), c)).alias(c) for c in acebmet_data.columns]).show()

Here, we can see there are no missing values in our data.

### Checking for Duplicate Values

In [27]:
acebmet_data.count()
acebmet_data.distinct().count()

127001

Here, we can see that the number of distinct values is the same as the number of values overall. Thus, we do not have any duplicate values.

## Prototype for automated script to aggregate meterological data

In [4]:
import os
rootdir = './input/raw'

file_list = []
for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        file_list.append(os.path.join(subdir, file))

### Creating lists of relevant .txt file names, separated into meteorological, nutrient, and water quality data

In [34]:
import re
text = [file for file in file_list if any(txt in file for txt in ['.txt'])]
regex = re.compile(r'.*(Readme).*|.*(checkpoint).*')
text_list = [i for i in text if not regex.match(i)]
text_list

met_list = [file for file in text_list if any(txt in file for txt in ['meteorological'])]
nut_list = [file for file in text_list if any(txt in file for txt in ['nutrient'])]
water_list = [file for file in text_list if any(txt in file for txt in ['water quality'])]

### Separating Meteorological file names by year

In [36]:
met_2004 = [file for file in met_list if any(txt in file for txt in ['2004'])]
met_2005 = [file for file in met_list if any(txt in file for txt in ['2005'])]
met_2006 = [file for file in met_list if any(txt in file for txt in ['2006'])]

### Concatenating Meteorological data of ALL locations for 2005 and 2006

In [55]:
met05_data = spark.read.option("header", "true") \
    .option("delimiter", "\t") \
    .option("inferSchema", "true") \
    .csv(met_2005)
met05_data = met05_data.drop(*['USRCODES'])

In [56]:
met06_data = spark.read.option("header", "true") \
    .option("delimiter", "\t") \
    .option("inferSchema", "true") \
    .csv(met_2006)
met06_data = met06_data.drop(*['USRCODES'])

### Checking for Colum

In [57]:
met06_data.columns == met05_data.columns

False

In [58]:
met05_data.select([count(when(isnan(c), c)).alias(c) for c in met05_data.columns]).show()

+-------+-----+--------+--------+-----+-------+--------+-------+--------+---+-----+------+-----+------+---+-----+------+-----+------+----+----+------+-------+--------+-------+--------+-------+------+-------+-------+--------+
|STNCODE|CLASS|SMPLDATE|SMPLTIME|ATemp|MaxTemp|MaxTempT|MinTemp|MinTempT| RH|MaxRH|MaxRHT|MinRH|MinRHT| BP|MaxBP|MaxBPT|MinBP|MinBPT|WSpd|Wdir|SDWDir|MaxWSpd|MaxWSpdT|MinWSpd|MinWSpdT|TotPrcp|TotPAR|AvgVolt|TotSRad|CummRain|
+-------+-----+--------+--------+-----+-------+--------+-------+--------+---+-----+------+-----+------+---+-----+------+-----+------+----+----+------+-------+--------+-------+--------+-------+------+-------+-------+--------+
|      0|    0|       0|       0|    0|      0|       0|      0|       0|  0|    0|     0|    0|     0|  0|    0|     0|    0|     0|   0|   0|     0|      0|       0|      0|       0|      0|     0|      0|      0|       0|
+-------+-----+--------+--------+-----+-------+--------+-------+--------+---+-----+------+-----+----

In [59]:
met06_data.select([count(when(isnan(c), c)).alias(c) for c in met06_data.columns]).show()

+-------+-----+--------+--------+-----+-------+--------+-------+--------+---+-----+------+-----+------+---+-----+------+-----+------+----+----+------+-------+--------+-------+--------+-------+------+-------+-------+
|STNCODE|CLASS|SMPLDATE|SMPLTIME|ATemp|MaxTemp|MaxTempT|MinTemp|MinTempT| RH|MaxRH|MaxRHT|MinRH|MinRHT| BP|MaxBP|MaxBPT|MinBP|MinBPT|WSpd|Wdir|SDWDir|MaxWSpd|MaxWSpdT|MinWSpd|MinWSpdT|TotPrcp|TotPAR|AvgVolt|DnIrrad|
+-------+-----+--------+--------+-----+-------+--------+-------+--------+---+-----+------+-----+------+---+-----+------+-----+------+----+----+------+-------+--------+-------+--------+-------+------+-------+-------+
|      0|    0|       0|       0|    0|      0|       0|      0|       0|  0|    0|     0|    0|     0|  0|    0|     0|    0|     0|   0|   0|     0|      0|       0|      0|       0|      0|     0|      0|      0|
+-------+-----+--------+--------+-----+-------+--------+-------+--------+---+-----+------+-----+------+---+-----+------+-----+------+---